In [1]:

from llama_index.core.retrievers import NLSQLRetriever

import sqlite3
import os
import llama_index
import sqlalchemy
import openai

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI


from sqlalchemy import text


In [2]:
import os
from sqlalchemy.orm import sessionmaker

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key


engine = create_engine("sqlite:///classes.db")

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
sql_database = SQLDatabase(engine, include_tables=["class_data"])


In [5]:
#Establishing engines and connections

conn = sqlite3.connect('courses_temp.db')
cursor = conn.cursor()


engine = create_engine("sqlite:///courses_temp.db")

llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")
sql_database = SQLDatabase(engine, include_tables=["class_data"])


In [5]:
#sqlite query
cursor.execute("PRAGMA table_info(class_data)")
tables = cursor.fetchall()
for col in tables:
    print(col[1], col[2])

course_campus TEXT
course_credits TEXT
course_description TEXT
course_id TEXT
course_offered TEXT
course_title TEXT
department_abbrev TEXT
is_bottleneck TEXT
is_gateway TEXT
prereq_string TEXT
course_coi REAL
course_level_coi REAL
curric_coi REAL
percent_in_range REAL
autumn INTEGER
winter INTEGER
spring INTEGER
summer INTEGER
0 REAL
1 REAL
2 REAL
3 REAL
4 REAL
5 REAL
6 REAL
7 REAL
8 REAL
9 REAL
10 REAL
11 REAL
12 REAL
13 REAL
14 REAL
15 REAL
16 REAL
17 REAL
18 REAL
19 REAL
20 REAL
21 REAL
22 REAL
23 REAL
24 REAL
25 REAL
26 REAL
27 REAL
28 REAL
29 REAL
30 REAL
31 REAL
32 REAL
33 REAL
34 REAL
35 REAL
36 REAL
37 REAL
38 REAL
39 REAL
40 REAL
mean REAL
median REAL
sd REAL
fourpointo_per REAL


In [17]:

#sqlalchemy query

with engine.connect() as con:
    rows = con.execute(text("SELECT course_title, mean_gpa\nFROM class_data\nWHERE department_abbrev = 'ANTH'\nAND course_credits = '4.0'\nORDER BY mean_gpa DESC\nLIMIT 4"))
    for row in rows:
        print(row)

('Hunter-Gatherer Societies', None)


In [6]:
#llama index retriever

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["class_data"], return_raw=True
)


In [24]:
#llama index query retriever


query = """
Schema Context:
Department_abbrev column represents the department and is abbreviated in all caps. Example: CS for Computer Science.
Course_credits contain floating points. Example: 4.0.

Choose one of these for urban: UDP, URBAN, DP, URBDP, URB
Question:
What are some urban design and planning courses?"""
 

response = nl_sql_retriever.retrieve(query)
print(response)


[NodeWithScore(node=TextNode(id_='6cd2987c-8cfc-48eb-a7d2-73bbfa1a1f5c', embedding=None, metadata={'sql_query': "SELECT course_title, course_description, course_credits \nFROM class_data \nWHERE department_abbrev = 'UDP' OR department_abbrev = 'URBAN' OR department_abbrev = 'DP' OR department_abbrev = 'URBDP' OR department_abbrev = 'URB'", 'result': [], 'col_keys': ['course_title', 'course_description', 'course_credits']}, excluded_embed_metadata_keys=['sql_query', 'result', 'col_keys'], excluded_llm_metadata_keys=['sql_query', 'result', 'col_keys'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='[]', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)]
